In [1]:
import requests
import json  
import pandas as pd
import numpy as np

In [2]:
response= requests.get('https://api.tfl.gov.uk/BikePoint/')
stations = response.json()

In [3]:
stations[0]

{'$type': 'Tfl.Api.Presentation.Entities.Place, Tfl.Api.Presentation.Entities',
 'id': 'BikePoints_1',
 'url': '/Place/BikePoints_1',
 'commonName': 'River Street , Clerkenwell',
 'placeType': 'BikePoint',
 'additionalProperties': [{'$type': 'Tfl.Api.Presentation.Entities.AdditionalProperties, Tfl.Api.Presentation.Entities',
   'category': 'Description',
   'key': 'TerminalName',
   'sourceSystemKey': 'BikePoints',
   'value': '001023',
   'modified': '2023-09-06T06:57:56.677Z'},
  {'$type': 'Tfl.Api.Presentation.Entities.AdditionalProperties, Tfl.Api.Presentation.Entities',
   'category': 'Description',
   'key': 'Installed',
   'sourceSystemKey': 'BikePoints',
   'value': 'true',
   'modified': '2023-09-06T06:57:56.677Z'},
  {'$type': 'Tfl.Api.Presentation.Entities.AdditionalProperties, Tfl.Api.Presentation.Entities',
   'category': 'Description',
   'key': 'Locked',
   'sourceSystemKey': 'BikePoints',
   'value': 'false',
   'modified': '2023-09-06T06:57:56.677Z'},
  {'$type': 'Tfl.

In [4]:
data =  pd.DataFrame()
s_names = []
s_lat = []
s_lon = []

NbBikes = []
NbEmptyDocks = []
NbDocks = []
NbStandardBikes = []
NbEBikes = []
s_id = []

In [5]:
len_lat = 0
for station in stations:
    s_names.append(station['commonName'])
    s_lat.append(float(station['lat']))
    s_lon.append(float(station['lon']))
    for add_property in station['additionalProperties']:
        if add_property['key'] == 'NbBikes':
            NbBikes.append(int(add_property['value']))
        if add_property['key'] == 'NbEmptyDocks':
            NbEmptyDocks.append(int(add_property['value']))     
        if add_property['key'] == 'NbDocks':
            NbDocks.append(int(add_property['value']))
        if add_property['key'] == 'NbStandardBikes':
            NbStandardBikes.append(int(add_property['value']))
        if add_property['key'] == 'NbEBikes':
            NbEBikes.append(int(add_property['value']))
        if add_property['key'] == 'TerminalName':
            s_id.append(int(add_property['value']))
        

In [6]:
len(s_lat)

796

In [7]:
data['Station_name'] = s_names
data['s_lat'] = s_lat
data['s_lon'] = s_lon
data['s_num_bikes'] = NbBikes
data['s_num_empty_docks'] = NbEmptyDocks
data['s_num_std_bikes'] = NbStandardBikes
data['s_num_e-bikes'] = NbEBikes
data['s_num_docks'] = data['s_num_bikes'] + data['s_num_empty_docks']
data['s_id'] = s_id

In [8]:
data

,Station_name,s_lat,s_lon,s_num_bikes,s_num_empty_docks,s_num_std_bikes,s_num_e-bikes,s_num_docks,s_id
0,"River Street , Clerkenwell",51.529163,-0.109970,0,19,0,0,19,1023
1,"Phillimore Gardens, Kensington",51.499606,-0.197574,14,23,14,0,37,1018
2,"Christopher Street, Liverpool Street",51.521283,-0.084605,28,4,25,3,32,1012
3,"St. Chad's Street, King's Cross",51.530059,-0.120973,13,10,13,0,23,1013
4,"Sedding Street, Sloane Square",51.493130,-0.156876,24,0,24,0,24,3420
...,...,...,...,...,...,...,...,...,...
791,"Crimscott Street, Bermondsey",51.495598,-0.078893,1,17,1,0,18,300061
792,"Brandon Street, Walworth",51.489102,-0.091548,6,15,6,0,21,300060
793,"The Blue, Bermondsey",51.492221,-0.062513,12,10,12,0,22,300059
794,"Coomer Place, West Kensington",51.483570,-0.202038,2,25,2,0,27,200081


In [9]:
# South East limits
se_limit_lat = 51.523811 # everthing lower than this value is south east
se_limit_lon = -0.101342 # everything greater than this value is south east

# East limits
e_limit_lat = 51.523811 # everything greater than this value is east
e_limit_lon = -0.022306 # everything greater than this is east

# South West limits
sw_limit_lat = 51.501631 # everthing lower than this value is south east
sw_limit_lon = -0.101342 # everything lower than this value is south east

# West limits
w_limit_lat = 51.501072  # everthing greater than this value is south east
w_limit_lon = -0.182406  # everthing lower than this value is south east

# Central limits
c_toplimit_lat = 51.501072 # everthing greater than this value is central
c_lowlimit_lat = 51.523811 # everything lower than this value is central

c_toplimit_lon = -0.101342 # everything lower than this value is central
c_lowlimit_lon = -0.182406 # everthing greater than this value is central

# North limits
n_toplimit_lon = -0.022306 # everything lower than this value is central
n_lowlimit_lon = -0.182406 # everthing greater than this value is central

n_lowlimit_lat = 51.523811 # everthing greater than this value is central

In [10]:
loc = []

In [11]:
for station in data.index:
    if data['s_lat'][station] > c_toplimit_lat and data['s_lat'][station]< c_lowlimit_lat and data['s_lon'][station]< c_toplimit_lon and data['s_lon'][station]>c_lowlimit_lon:
        loc.append('Central')
    elif data['s_lon'][station] < n_toplimit_lon and data['s_lon'][station] > n_lowlimit_lon and data['s_lat'][station] > n_lowlimit_lat:
        loc.append('North')
    elif data['s_lon'][station] < w_limit_lon and data['s_lat'][station] > w_limit_lat:
        loc.append('West')
    elif data['s_lon'][station] < sw_limit_lon and data['s_lat'][station] < sw_limit_lat:
        loc.append('South_West')  
    elif data['s_lon'][station] > e_limit_lon and data['s_lat'][station] > e_limit_lat:
        loc.append('East')     
    elif data['s_lon'][station] > se_limit_lon and data['s_lat'][station] < se_limit_lat:
        loc.append('East')    
    else:
        loc.append('Other')

In [12]:
data['Area_loc'] = loc

In [13]:
data

,Station_name,s_lat,s_lon,s_num_bikes,s_num_empty_docks,s_num_std_bikes,s_num_e-bikes,s_num_docks,s_id,Area_loc
0,"River Street , Clerkenwell",51.529163,-0.109970,0,19,0,0,19,1023,North
1,"Phillimore Gardens, Kensington",51.499606,-0.197574,14,23,14,0,37,1018,South_West
2,"Christopher Street, Liverpool Street",51.521283,-0.084605,28,4,25,3,32,1012,East
3,"St. Chad's Street, King's Cross",51.530059,-0.120973,13,10,13,0,23,1013,North
4,"Sedding Street, Sloane Square",51.493130,-0.156876,24,0,24,0,24,3420,South_West
...,...,...,...,...,...,...,...,...,...,...
791,"Crimscott Street, Bermondsey",51.495598,-0.078893,1,17,1,0,18,300061,East
792,"Brandon Street, Walworth",51.489102,-0.091548,6,15,6,0,21,300060,East
793,"The Blue, Bermondsey",51.492221,-0.062513,12,10,12,0,22,300059,East
794,"Coomer Place, West Kensington",51.483570,-0.202038,2,25,2,0,27,200081,South_West


In [14]:
stations_details=data['Station_name'].str.split(', ', expand=True)
stations=stations_details.rename(columns={0:'Street',1:'Neighbourhood'})

In [15]:
stations_fv=stations[['Street','Neighbourhood']]
stations_fv['Street']=stations_fv['Street'].apply(lambda x: x.strip())
# stations_fv['Neighbourhood']=stations_fv['Neighbourhood'].apply(lambda x: x.strip())

/tmp/ipykernel_13875/2823504512.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations_fv['Street']=stations_fv['Street'].apply(lambda x: x.strip())


In [16]:
stations_fv['Neighbourhood'].unique()

array(['Clerkenwell', 'Kensington', 'Liverpool Street', "King's Cross",
       'Sloane Square', 'Marylebone', "St. John's Wood", 'Maida Vale',
       'Bankside', 'Bloomsbury', 'Fitzrovia', 'Kings Cross', 'Holborn',
       'Covent Garden', 'Euston', 'Hackney Central', 'Somers Town',
       'Temple', 'Bayswater', 'Hoxton', 'Shoreditch', 'Aldgate',
       "Earl's Court", 'Mayfair', 'Wapping', "St. Paul's", 'Finsbury',
       "St. Luke's", 'Barbican', 'The Borough', 'Kennington', 'Strand',
       'Vauxhall', 'Angel', "The Regent's Park", 'Southwark', 'West End',
       'Bermondsey', 'Camden Town', 'Elephant & Castle', 'Borough',
       'Bank', 'Tower', 'Westminster', 'Soho', 'Hyde Park',
       'South Kensington', 'Guildhall', 'Belgravia', 'Moorgate',
       'Knightsbridge', 'Pimlico', 'Victoria', 'Oval', 'Notting Hill',
       'Walworth', 'Waterloo', "St. James's", 'Paddington', 'South Bank',
       'Nine Elms', 'Monument', 'Farringdon', 'Whitechapel', 'Chelsea',
       'Kensington Garden

In [17]:
stations_fv['Station_name']=data['Station_name']

In [18]:
df_stations=data.merge(stations_fv,on='Station_name')

In [19]:
df_stations

,Station_name,s_lat,s_lon,s_num_bikes,s_num_empty_docks,s_num_std_bikes,s_num_e-bikes,s_num_docks,s_id,Area_loc,Street,Neighbourhood
0,"River Street , Clerkenwell",51.529163,-0.109970,0,19,0,0,19,1023,North,River Street,Clerkenwell
1,"Phillimore Gardens, Kensington",51.499606,-0.197574,14,23,14,0,37,1018,South_West,Phillimore Gardens,Kensington
2,"Christopher Street, Liverpool Street",51.521283,-0.084605,28,4,25,3,32,1012,East,Christopher Street,Liverpool Street
3,"St. Chad's Street, King's Cross",51.530059,-0.120973,13,10,13,0,23,1013,North,St. Chad's Street,King's Cross
4,"Sedding Street, Sloane Square",51.493130,-0.156876,24,0,24,0,24,3420,South_West,Sedding Street,Sloane Square
...,...,...,...,...,...,...,...,...,...,...,...,...
791,"Crimscott Street, Bermondsey",51.495598,-0.078893,1,17,1,0,18,300061,East,Crimscott Street,Bermondsey
792,"Brandon Street, Walworth",51.489102,-0.091548,6,15,6,0,21,300060,East,Brandon Street,Walworth
793,"The Blue, Bermondsey",51.492221,-0.062513,12,10,12,0,22,300059,East,The Blue,Bermondsey
794,"Coomer Place, West Kensington",51.483570,-0.202038,2,25,2,0,27,200081,South_West,Coomer Place,West Kensington


In [20]:
df_stations.columns

Index(['Station_name', 's_lat', 's_lon', 's_num_bikes', 's_num_empty_docks',
       's_num_std_bikes', 's_num_e-bikes', 's_num_docks', 's_id', 'Area_loc',
       'Street', 'Neighbourhood'],
      dtype='object')

In [39]:
stations_df_st=df_stations[['s_id','Station_name','Street',
                            'Neighbourhood','Area_loc', 's_lat', 's_lon','s_num_docks']]

In [40]:
stations_df_st

,s_id,Station_name,Street,Neighbourhood,Area_loc,s_lat,s_lon,s_num_docks
0,1023,"River Street , Clerkenwell",River Street,Clerkenwell,North,51.529163,-0.109970,19
1,1018,"Phillimore Gardens, Kensington",Phillimore Gardens,Kensington,South_West,51.499606,-0.197574,37
2,1012,"Christopher Street, Liverpool Street",Christopher Street,Liverpool Street,East,51.521283,-0.084605,32
3,1013,"St. Chad's Street, King's Cross",St. Chad's Street,King's Cross,North,51.530059,-0.120973,23
4,3420,"Sedding Street, Sloane Square",Sedding Street,Sloane Square,South_West,51.493130,-0.156876,24
...,...,...,...,...,...,...,...,...
791,300061,"Crimscott Street, Bermondsey",Crimscott Street,Bermondsey,East,51.495598,-0.078893,18
792,300060,"Brandon Street, Walworth",Brandon Street,Walworth,East,51.489102,-0.091548,21
793,300059,"The Blue, Bermondsey",The Blue,Bermondsey,East,51.492221,-0.062513,22
794,200081,"Coomer Place, West Kensington",Coomer Place,West Kensington,South_West,51.483570,-0.202038,27


### Colour Formatting

In [41]:
# color_dict={'Area_loc': list(stations_df_st['Area_loc'].unique())
#                        , 'Colours': ['#ffbaba','#ff7b7b', '#ff5252', '#ff0000', '#a70000']
#                     ,'index':[0, 1, 2, 3,4]}

# color_df=pd.DataFrame(color_dict)
# color_df


In [42]:
# stations_df_st.columns

In [43]:
# stations_df_st=stations_df_st.merge(color_df,on='Area_loc',how='left')

In [44]:
# stations_df_st

### Size Bucket

In [45]:

stations_df_st['Size_bucket']=[(50 if x<=15 else (100 if 15<x<=30 else (150 if 30<x<=45 else 200))) for x in stations_df_st['s_num_docks']]



/tmp/ipykernel_13875/1678009662.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations_df_st['Size_bucket']=[(50 if x<=15 else (100 if 15<x<=30 else (150 if 30<x<=45 else 200))) for x in stations_df_st['s_num_docks']]


In [46]:
path_st='../../raw_data/stations_df_st.csv'

In [47]:
stations_df_st.to_csv(path_st)

In [48]:
stations_df_st

,s_id,Station_name,Street,Neighbourhood,Area_loc,s_lat,s_lon,s_num_docks,Size_bucket
0,1023,"River Street , Clerkenwell",River Street,Clerkenwell,North,51.529163,-0.109970,19,100
1,1018,"Phillimore Gardens, Kensington",Phillimore Gardens,Kensington,South_West,51.499606,-0.197574,37,150
2,1012,"Christopher Street, Liverpool Street",Christopher Street,Liverpool Street,East,51.521283,-0.084605,32,150
3,1013,"St. Chad's Street, King's Cross",St. Chad's Street,King's Cross,North,51.530059,-0.120973,23,100
4,3420,"Sedding Street, Sloane Square",Sedding Street,Sloane Square,South_West,51.493130,-0.156876,24,100
...,...,...,...,...,...,...,...,...,...
791,300061,"Crimscott Street, Bermondsey",Crimscott Street,Bermondsey,East,51.495598,-0.078893,18,100
792,300060,"Brandon Street, Walworth",Brandon Street,Walworth,East,51.489102,-0.091548,21,100
793,300059,"The Blue, Bermondsey",The Blue,Bermondsey,East,51.492221,-0.062513,22,100
794,200081,"Coomer Place, West Kensington",Coomer Place,West Kensington,South_West,51.483570,-0.202038,27,100


In [49]:
# stations_df_st[stations_df_st.Neighbourhood=='Liverpool Street']

### Changing Station's Names

In [50]:
stations_df_st['stations']=stations_df_st['Station_name'].apply(lambda x: x.strip().lower().replace(',',' ').replace('.','').replace('(','').replace(')','').replace('&','').replace(' ','_').replace("'",""))

/tmp/ipykernel_13875/2332289802.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations_df_st['stations']=stations_df_st['Station_name'].apply(lambda x: x.strip().lower().replace(',',' ').replace('.','').replace('(','').replace(')','').replace('&','').replace(' ','_').replace("'",""))


In [51]:
stations_df_st

,s_id,Station_name,Street,Neighbourhood,Area_loc,s_lat,s_lon,s_num_docks,Size_bucket,stations
0,1023,"River Street , Clerkenwell",River Street,Clerkenwell,North,51.529163,-0.109970,19,100,river_street___clerkenwell
1,1018,"Phillimore Gardens, Kensington",Phillimore Gardens,Kensington,South_West,51.499606,-0.197574,37,150,phillimore_gardens__kensington
2,1012,"Christopher Street, Liverpool Street",Christopher Street,Liverpool Street,East,51.521283,-0.084605,32,150,christopher_street__liverpool_street
3,1013,"St. Chad's Street, King's Cross",St. Chad's Street,King's Cross,North,51.530059,-0.120973,23,100,st_chads_street__kings_cross
4,3420,"Sedding Street, Sloane Square",Sedding Street,Sloane Square,South_West,51.493130,-0.156876,24,100,sedding_street__sloane_square
...,...,...,...,...,...,...,...,...,...,...
791,300061,"Crimscott Street, Bermondsey",Crimscott Street,Bermondsey,East,51.495598,-0.078893,18,100,crimscott_street__bermondsey
792,300060,"Brandon Street, Walworth",Brandon Street,Walworth,East,51.489102,-0.091548,21,100,brandon_street__walworth
793,300059,"The Blue, Bermondsey",The Blue,Bermondsey,East,51.492221,-0.062513,22,100,the_blue__bermondsey
794,200081,"Coomer Place, West Kensington",Coomer Place,West Kensington,South_West,51.483570,-0.202038,27,100,coomer_place__west_kensington


### List of interesting stations

In [60]:
interesting_stations=['eagle_wharf_road__hoxton','graham_street__angel','finsbury_circus__liverpool_street','wormwood_street__liverpool_street',
                      'lancaster_gate___bayswater','devonshire_terrace__bayswater']

In [59]:
closest_station='Devonshire Terrace, Bayswater'.strip().lower().replace(',',' ').replace('.','').replace('(','').replace(')','').replace('&','').replace(' ','_').replace("'","")
closest_station

'devonshire_terrace__bayswater'

In [61]:
interesting_stations=['eagle_wharf_road__hoxton','wenlock_road__hoxton','finsbury_circus__liverpool_street','wormwood_street__liverpool_street', 'lancaster_gate___bayswater','devonshire_terrace__bayswater','graham_street__angel']

In [62]:
missing_stations=['wormwood_street__liverpool_street','devonshire_terrace__bayswater','graham_street__angel']

## Calling the API

In [41]:
response_2= requests.get('https://api.tfl.gov.uk/BikePoint/')
stations_2 = response_2.json()

In [48]:
stations_2[0]

{'$type': 'Tfl.Api.Presentation.Entities.Place, Tfl.Api.Presentation.Entities',
 'id': 'BikePoints_1',
 'url': '/Place/BikePoints_1',
 'commonName': 'River Street , Clerkenwell',
 'placeType': 'BikePoint',
 'additionalProperties': [{'$type': 'Tfl.Api.Presentation.Entities.AdditionalProperties, Tfl.Api.Presentation.Entities',
   'category': 'Description',
   'key': 'TerminalName',
   'sourceSystemKey': 'BikePoints',
   'value': '001023',
   'modified': '2023-09-01T21:08:07.183Z'},
  {'$type': 'Tfl.Api.Presentation.Entities.AdditionalProperties, Tfl.Api.Presentation.Entities',
   'category': 'Description',
   'key': 'Installed',
   'sourceSystemKey': 'BikePoints',
   'value': 'true',
   'modified': '2023-09-01T21:08:07.183Z'},
  {'$type': 'Tfl.Api.Presentation.Entities.AdditionalProperties, Tfl.Api.Presentation.Entities',
   'category': 'Description',
   'key': 'Locked',
   'sourceSystemKey': 'BikePoints',
   'value': 'false',
   'modified': '2023-09-01T21:08:07.183Z'},
  {'$type': 'Tfl.

In [46]:
origin='Shoreditch Park, Hoxton'
destination='Finsbury Circus, Liverpool Street'

In [89]:
for station in stations_2:
    if station['commonName']==origin:
        for add_property in station['additionalProperties']:
            if add_property['key'] == 'NbBikes':
                print(add_property['value'])
    if station['commonName']==destination:
            for add_property in station['additionalProperties']:
                if add_property['key'] == 'NbEmptyDocks':
                    print(add_property['value'])
        


12
18


In [93]:
for station in stations_2:
    if station['commonName']==origin:
        print(station['lat'])
        print(station['lon'])
    if station['commonName']==destination:
        print(station['lat'])
        print(station['lon'])
        

51.517075
-0.086685
51.534042
-0.086379
